In [ ]:
from initParam import *

# Drift Equilibrium
Fixed: β, r

Eq: V, Fx, δ

In [ ]:
# parameters and initial conditions
FORCE_REAR = 9.0 # [N] constant rear longitudinal force per wheel
STEER = np.deg2rad(15) # [rad] constant steering angle
DELTA, FX = STEER * steering_ratio_f, 2 * FORCE_REAR # wheel angle and longitudinal force
U0, V0, R0 = .5, 10.0, 1.0 # initial conditions


T = 3 # [s] total simulation time
DT = 0.001 # [s] time step for the simulation


In [ ]:
# equation of motion for the Single Track Model (power to rear only)
def d_uvr(uvr, δ, Fx): 
    u, v, r = uvr # unpack the state vector
    αf = δ - arctan2(v + a*r, u) # slip angle front
    αr = -arctan2(v - b*r, u) # slip angle rear
    Fyf = tire(αf, 0.0, Fz_Front, μf, Cyf) # lateral force front
    Fyr = tire(αr, Fx, Fz_Rear, μr, Cyr) # lateral force rear
    Fxr = Fx # rear longitudinal force
    return np.array([ # equations of motion
        (Fxr - Fyf * sin(δ)) / m + r * v,
        (Fyf * cos(δ) + Fyr) / m - r * u,
        (a * Fyf * cos(δ) - b * Fyr) / J_CoG
    ])

def stm_rk4(uvr, δ, Fx, dt=DT): # runge-kutta 4th order method
    k1 = d_uvr(uvr, δ, Fx) * dt
    k2 = d_uvr(uvr + k1/2, δ, Fx) * dt
    k3 = d_uvr(uvr + k2/2, δ, Fx) * dt
    k4 = d_uvr(uvr + k3, δ, Fx) * dt
    return uvr + (k1 + 2*k2 + 2*k3 + k4) / 6 # update the state vector

# simulate the STM for x seconds
n_steps = int(T / DT) # number of steps in the simulation
# initialize the state vector
state = np.zeros((n_steps, 3)) # [u, v, r]
state[0] = np.array([U0, V0, R0]) # initial conditions
print(f"Initial state: {state[0]}") # print the initial state
# initialize the control input vector
control = np.zeros((n_steps, 2)) # [Fx, δ]
control[:,0], control[:,1] = FX, DELTA # assign constant values
# run the simulation
for i in range(1, n_steps):
    state[i] = stm_rk4(state[i-1], control[i, 1], control[i, 0])  

# plot first n steps of the simulation  
dec = 10 # decimation factor
u1,v1,r1 = state[:,0], state[:,1], state[:,2] # unpack the state vector
u1, v1, r1 = u1[::dec], v1[::dec], r1[::dec]
plt.figure(figsize=(8, 3))
plt.plot(u1, label='u1 (longitudinal velocity)')
plt.plot(v1, label='v1 (lateral velocity)')
plt.plot(r1, label='r1 (yaw rate)')
plt.title('STM Simulation Results')
plt.xlabel('Time Step')
plt.ylabel('State Variables')
plt.legend()
plt.tight_layout()
plt.show()

# print the final state
print(f"Final state: {state[-1]}")

In [ ]:
# equation of motion for the Single Track Model with beta change of variable
def d_vβr(vβr, δ, Fx): 
    v, β, r = vβr # unpack the state vector
    Fyf = tire(f_αf(δ,v,β,r), 0.0, Fz_Front, μf, Cyf) # lateral force front
    Fyr = tire(f_αr(δ,v,β,r), Fx, Fz_Rear, μr, Cyr) # lateral force rear
    Fxr = Fx # rear longitudinal force
    return np.array([ # equations of motion
        (-Fyf*sin(δ-β) + Fxr*cos(β) + Fyr*sin(β)) / m, # V dot
        (+Fyf*cos(δ-β) - Fxr*sin(β) + Fyr*cos(β)) / (m*v) - r, # β dot
        (a*Fyf*cos(δ) - b*Fyr) / J_CoG # r dot
    ])

def stm_rk4(vβr, δ, Fx, dt=DT): # runge-kutta 4th order method
    k1 = d_vβr(vβr, δ, Fx) * dt
    k2 = d_vβr(vβr + k1/2, δ, Fx) * dt
    k3 = d_vβr(vβr + k2/2, δ, Fx) * dt
    k4 = d_vβr(vβr + k3, δ, Fx) * dt
    return vβr + (k1 + 2*k2 + 2*k3 + k4) / 6 # update the state vector

# simulate the STM for x seconds
n_steps = int(T / DT) # number of steps in the simulation
# initialize the state vector
state = np.zeros((n_steps, 3)) # [u, v, r]
state[0] = vel2beta(np.array([U0, V0, R0])) # initial state vector
print(f"Initial state: {state[0]} (v,β,r)") # print the initial state in v,β,r format
print(f"Initial state: {beta2vel(state[0])} (u,v,r)") # print the initial state

# initialize the control input vector
control = np.zeros((n_steps, 2)) # [Fx, δ]
control[:,0], control[:,1] = FX, DELTA # assign constant values
# run the simulation
for i in range(1, n_steps):
    state[i] = stm_rk4(state[i-1], control[i, 1], control[i, 0])  

# plot first n steps of the simulation  
dec = 10 # decimation factor
v1, β1, r1 = state[:,0], state[:,1], state[:,2] # unpack the state vector
v1, β1, r1 = v1[::dec], β1[::dec], r1[::dec] # decimate the state vector
plt.figure(figsize=(8, 3))
plt.plot(v1, label='v1 (velocity)')
plt.plot(β1, label='β1 (side slip angle)')
plt.plot(r1, label='r1 (yaw rate)')
plt.title('STM Simulation Results')
plt.xlabel('Time Step')
plt.ylabel('State Variables')
plt.legend()
plt.tight_layout()
plt.show()

# uvr = np.array([beta2vel(s) for s in state]) # convert state to u,v,r format
uvr = beta2vel(state) # convert state to u,v,r format
u1, v1, r1 = uvr[:,0], uvr[:,1], uvr[:,2] # unpack the state vector
u1, v1, r1 = u1[::dec], v1[::dec], r1[::dec] # decimate the state vector
plt.figure(figsize=(8, 3))
plt.plot(u1, label='u1 (longitudinal velocity)')
plt.plot(v1, label='v1 (lateral velocity)')
plt.plot(r1, label='r1 (yaw rate)')
plt.title('STM Simulation Results')
plt.xlabel('Time Step')
plt.ylabel('State Variables')
plt.legend()
plt.tight_layout()
plt.show()

# print the final state
print(f"Final state: {state[-1]} (v,β,r)") # print the final state in v,β,r format
print(f"Final state: {beta2vel(state[-1])} (u,v,r)") # print the final state in u,v,r format

In [ ]:
# calculate the gradient of the state vector on a 3d grid
n = 50 # 100
us = np.linspace(5, 15, n) # longitudinal velocity
vs = np.linspace(-10, 10, n) # lateral velocity
rs = np.linspace(-2, 2, n) # yaw rate

duvr = np.zeros((len(us), len(vs), len(rs), 3)) # gradient of the state vector
for i, u in enumerate(tqdm(us)):
    for j, v in enumerate(vs):
        for k, r in enumerate(rs):
            duvr[i, j, k] = d_uvr(np.array([u, v, r]), DELTA, FX) # calculate the gradient at each point

norm_duvr = np.linalg.norm(duvr, axis=-1) # calculate the norm of the gradient
min_norm, max_norm = norm_duvr.min(), norm_duvr.max()
print(f'Max: {max_norm:.2f}, Min: {min_norm:.2f}')

In [ ]:
# plot slices of the gradient
LEV = 30 # number of contour levels
# 1 - slicing over u
idxs = np.linspace(0, len(us) - 1, 25, dtype=int)
plt.figure(figsize=(15, 15))
for ip, i in enumerate(idxs): # plot every 10th slice
    plt.subplot(5, 5, ip+1)
    # plot contour of norm_duvr with fixed min/max values
    contour = plt.contour(vs, np.rad2deg(rs), norm_duvr[i, :, :], levels=LEV, vmin=min_norm, vmax=max_norm, cmap=CM)
    plt.title(f'u = {us[i]:.2f} m/s')
    # plt.colorbar(contour)
    if ip>=20: plt.xlabel('Lateral Velocity v [m/s]')
    if ip%5 == 0: plt.ylabel('Yaw Rate r [deg/s]')
plt.suptitle(f'Fx = {FX:.2f} N, δ = {np.rad2deg(DELTA):.2f} deg')
plt.tight_layout()
plt.show()

# 2 - slicing over v
idxs = np.linspace(0, len(vs) - 1, 25, dtype=int)
plt.figure(figsize=(15, 15))
for ip, i in enumerate(idxs): # plot every 10th slice
    plt.subplot(5, 5, ip+1)
    # plot contour of norm_duvr with fixed min/max values
    contour = plt.contour(us, np.rad2deg(rs), norm_duvr[:, i, :], levels=LEV, vmin=min_norm, vmax=max_norm, cmap=CM)
    plt.title(f'v = {vs[i]:.2f} m/s')
    # plt.colorbar(contour)
    if ip>=20: plt.xlabel('Longitudinal Velocity u [m/s]')
    if ip%5 == 0: plt.ylabel('Yaw Rate r [deg/s]')
plt.suptitle(f'Fx = {FX:.2f} N, δ = {np.rad2deg(DELTA):.2f} deg')
plt.tight_layout()
plt.show()  
# 3 - slicing over r
idxs = np.linspace(0, len(rs) - 1, 25, dtype=int)
plt.figure(figsize=(15, 15))
for ip, i in enumerate(idxs): # plot every 10th slice
    plt.subplot(5, 5, ip+1)
    # plot contour of norm_duvr with fixed min/max values
    contour = plt.contour(us, vs, norm_duvr[:, :, i], levels=LEV, vmin=min_norm, vmax=max_norm, cmap=CM)
    plt.title(f'r = {np.rad2deg(rs[i]):.2f} deg/s')
    # plt.colorbar(contour)
    if ip>=20: plt.xlabel('Longitudinal Velocity u [m/s]')
    if ip%5 == 0: plt.ylabel('Lateral Velocity v [m/s]')
plt.suptitle(f'Fx = {FX:.2f} N, δ = {np.rad2deg(DELTA):.2f} deg')
plt.tight_layout()
plt.show()

In [ ]:
# gradient descent to find equilibrium point, fixed Fx and δ
tol = 1e-8
lr = 1e-1
uvr0 = np.array([9, -3, np.deg2rad(50)]) # initial guess for the equilibrium point
uvr = uvr0.copy() # copy the initial guess
plt.figure()
ax = plt.subplot(111, projection='3d')
ax.set_xlabel('u (longitudinal velocity)')
ax.set_ylabel('v (lateral velocity)')
ax.set_zlabel('r (yaw rate)')
ax.set_title('Gradient Descent Trajectory')
# ax.legend()

uvrs, duvrs = [], []

for i in tqdm(range(1000)): # maximum number of iterations
    ax.scatter(uvr[0], uvr[1], uvr[2], color='red', s=10)
    duvr = d_uvr(uvr, STEER, FX) # calculate the gradient
    r = uvr[2] # yaw rate
    if np.linalg.norm(duvr) < tol: # check for convergence
        print(f'Converged in {i} iterations')
        break
    uvr += duvr * lr # update the equilibrium point using a learning rate of 0.1
    uvr[2] = r # keep the yaw rate constant
    uvrs.append(uvr.copy()) # store the equilibrium point
    duvrs.append(duvr.copy()) # store the gradient
    # print(f'Iteration {i}: uvr = {uvr}, duvr = {duvr}')
ax.scatter(uvr[0], uvr[1], uvr[2], color='green', s=50, label='Equilibrium Point')
print(f'Equilibrium Point: u = {uvr[0]:.3f} [m/s], v = {uvr[1]:.3f} [m/s], r = {uvr[2]:.3f} [rad/s]')
print(f'uvr = [{uvr[0]:.3f}, {uvr[1]:.3f}, {uvr[2]:.3f}]; % uvr from eq calculation')
ax.view_init(elev=20, azim=30)
plt.tight_layout()
plt.show()

# plot the trajectory of the gradient descent
uvrs, duvrs = np.abs(np.array(uvrs)), np.abs(np.array(duvrs))
plt.figure()
plt.subplot(2, 1, 1)
plt.plot(uvrs)
plt.yscale('log')
plt.legend(['u', 'v', 'r'])
plt.subplot(2, 1, 2)
plt.plot(duvrs)
plt.yscale('log')
plt.legend(['du', 'dv', 'dr'])
plt.show()

In [ ]:
# same as before, but using casadi for symbolic differentiation
import casadi as ca
u = ca.SX.sym('u') # longitudinal velocity
v = ca.SX.sym('v') # lateral velocity
r = ca.SX.sym('r') # yaw rate
δ = ca.SX.sym('δ') # steering angle
Fx = ca.SX.sym('Fx') # longitudinal force


# define the symbolic state vector
uvr = ca.vertcat(u, v, r) # state vector
# define the symbolic control input vector
control = ca.vertcat(Fx, δ) # control input vector 
# define the symbolic tire model
αf = δ - ca.atan2(v + a*r, u) # slip angle front
αr = -ca.atan2(v - b*r, u) # slip angle rear
def ca_fiala_tanh(α, Fx, Fz, μ, Cy):
    Fy_max = ca.sqrt(μ**2 * Fz**2 - Fx**2) # maximum lateral force
    αs = ca.atan(Fy_max/Cy) # maximum slip angle
    return Fy_max * ca.tanh(α / αs) # tanh approximation

# define the symbolic lateral forces
Fyf = ca_fiala_tanh(αf, 0.0, Fz_Front, μf, Cyf) # lateral force front
Fyr = ca_fiala_tanh(αr, Fx, Fz_Rear, μr, Cyr) # lateral force rear
Fxr = Fx # rear longitudinal force

# define the symbolic equations of motion
du = (Fxr - Fyf * ca.sin(δ)) / m + r * v
dv = (Fyf * ca.cos(δ) + Fyr) / m - r * u
dr = (a * Fyf * ca.cos(δ) - b * Fyr) / J_CoG

duvr = ca.vertcat(du, dv, dr) # equations of motion

In [ ]:
# test casadi integration
ode = {}
ode['x'] = uvr # state vector
ode['p'] = ca.vertcat(Fx, δ) # control input vector
ode['ode'] = duvr # equations of motion
F = ca.integrator('F', 'cvodes', ode, 0, T) # create the integrator
print(f'ODE: {ode}')
print(f'U0: {U0}, V0: {V0}, R0: {R0}, T: {T}, DT: {DT}')
res = F(x0=[U0,V0,R0], p=[FX, DELTA])  # simulate the STM for x seconds
print(f'Final state: {res["xf"]}')  # print the result

In [ ]:
# test finding zeros of the equations of motion, using casadi
print(f'eqs: {duvr}')

fixed_r = 0.26 # [rad/s] fixed yaw rate

# substitute Fx and δ with their values
duvr_func = ca.substitute(duvr, Fx, FX) # substitute Fx and δ with their values
duvr_func = ca.substitute(duvr_func, δ, DELTA) # substitute Fx and δ with their values
duvr_func = ca.substitute(duvr_func, r, fixed_r) # substitute r with its value
duvr_func = ca.Function('duvr_func', [uvr], [duvr_func]) 

# Define optimization problem to find steady-state (duvr == 0)
w = ca.vertcat(u, v, r)
g = duvr_func(w)

# Define NLP
nlp = {'x': w, 'f': ca.sumsqr(g)}  # minimize sum of squares of residuals

# Create solver
solver = ca.nlpsol('solver', 'ipopt', nlp)

# Solve
sol = solver(x0=[1.0, 8.0, 0.26])  # Initial guess

print(f'Solution: {sol["x"]}')  # Print solution
print(f'Function value (residuals): {sol["f"]}')  # Print function value


# see if it somewhat constant
F = ca.integrator('F', 'cvodes', ode, 0, 0.5) # create the integrator
res = F(x0=sol["x"], p=[FX, DELTA])  # simulate the STM for x seconds
print(f'Final state: {res["xf"]}')  # print the result